# Requirements: 

1. 99.4% validation accuracy
2. Less than 20k Parameters
3. Less than 20 Epochs
4. Have used BN, Dropout, a Fully connected layer, have used GAP. 

# Imports

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

# Network

In [2]:
class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    
    self.block1 = nn.Sequential(
        nn.Conv2d(1, 16, 3),                                                    # (28, 28, 1)  --> (3, 3, 1, 16)  --> (26, 26, 16)  [R.F. 3]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 
        nn.Dropout2d(0.1), 

        nn.Conv2d(16, 16, 3),                                                   # (26, 26, 16) --> (3, 3, 16, 16) --> (24, 24, 16)  [R.F. 5]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 
        nn.Dropout2d(0.1),

        nn.Conv2d(16, 32, 3),                                                   # (24, 24, 16) --> (3, 3, 16, 32) --> (22, 22, 32)  [R.F. 7]
        nn.ReLU(), 
        nn.BatchNorm2d(32), 
        nn.Dropout2d(0.1)
        )
    
    self.block2 = nn.Sequential(
        nn.Conv2d(32, 16, 1),                                                   # (22, 22, 32) --> (1, 1, 32, 16) --> (22, 22, 16)  [R.F. 7]
        nn.ReLU(),
        nn.MaxPool2d(2,2),                                                      # (22, 22, 16) --> (2, 2)         --> (11, 11, 16)  [R.F. 14]

        nn.Conv2d(16, 16, 3),                                                   # (11, 11, 16) --> (3, 3, 16, 16) --> (9, 9, 16)    [R.F. 16]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 
        nn.Dropout2d(0.1)
    ) 

    self.block3 = nn.Sequential(
        nn.Conv2d(16, 16, 3),                                                   # (9, 8, 16)  --> (3, 3, 16, 16)  --> (7, 7, 16)     [R.F. 18]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 
        nn.Dropout2d(0.1),

        nn.Conv2d(16, 16, 3, padding=1),                                        # (7, 7, 16)  --> (3, 3, 16, 16)  --> (7, 7, 16)     [R.F. 20]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 
        nn.Dropout2d(0.1)
    )

    self.block4 = nn.Sequential(
        nn.Conv2d(16, 16, 3),                                                   # (7, 7, 16)  --> (3, 3, 16, 16)  --> (5, 5, 16)     [R.F. 22]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 

        nn.Conv2d(16, 16, 3, padding=1),                                        # (5, 5, 16)  --> (3, 3, 16, 16)  --> (5, 5, 16)     [R.F. 24]
        nn.ReLU(), 
        nn.BatchNorm2d(16), 
    ) 
    
    self.globalaveragepooling = nn.Sequential(
        nn.Conv2d(16, 10, 1),                                                   # (5, 5, 16)  --> (1, 1, 16, 10)  --> (5, 5, 10)     [R.F. 26]
        nn.AvgPool2d(5)
    )

    self.fc = nn.Sequential(
        nn.Linear(in_features=10, out_features=10)                              # Fully-connected network
    ) 

  def forward(self, x):

    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.globalaveragepooling(x)

    x = x.view(-1, 10)
    x = self.fc(x)

    return F.log_softmax(x)


In [3]:
!pip install torchsummary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
           Conv2d-13           [-1, 16, 22, 22]             528
             ReLU-14           [-1, 16,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return 100. * correct / len(test_loader.dataset)

# Training

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):

    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

    # Break if required test accuracy is achieved
    if accuracy >= 99.4:
      break 

loss=0.13771481812000275 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.76it/s]



Test set: Average loss: 0.0565, Accuracy: 9832/10000 (98.32%)



loss=0.05002450570464134 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.83it/s]



Test set: Average loss: 0.0346, Accuracy: 9893/10000 (98.93%)



loss=0.058828312903642654 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.27it/s]


...


Test set: Average loss: 0.0199, Accuracy: 9934/10000 (99.34%)



loss=0.009174360893666744 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.53it/s]



Test set: Average loss: 0.0213, Accuracy: 9932/10000 (99.32%)



loss=0.04397236928343773 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.71it/s]



Test set: Average loss: 0.0184, Accuracy: 9943/10000 (99.43%)

